# Traitement de données pour la régression sur la prédiction d'une seule station

## Interpolation

Essayer H1700010. Puisque les données de météo SYNOP sont à chaque 3h, et les donnée de débit sont à chaque 2h, on voudrait interpoler les deux données de sortes qu'elles sont tous à chaque 1h. On utilise d'abord l'interpolation linéaire.



In [2]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import os

In [4]:
for i in range(10,21):
    data = pd.read_csv("../Data/Extraction_Hydro/Seine/H1700010_20"+ str(i) +".csv", encoding="utf-8", sep=";")
    data["Date"]= data["Date"].str.split(pat="+",expand = True)
    data = data[["Date","Q (m3/s)"]]
    data["Date"]= pd.to_datetime(data["Date"])
    data = data.sort_values(by="Date")
    data= data.drop_duplicates("Date")
    data = data.set_index('Date').resample('1H')
    interpolated = data.interpolate(method='linear')
    interpolated.to_csv("../Data/Extraction_Hydro/Seine/H1700010_20"+ str(i) +"_ip.csv", header = True)

In [33]:
filepaths = ["../Data/Extraction_Hydro/Seine/H1700010_20"+ str(i) +"_ip.csv" for i in range(10,20)]
data = pd.concat(map(pd.read_csv, filepaths))
#data.to_csv("../Data/Extraction_Hydro/Seine/H1700010_ip.csv", header = True)

Il me semble que les données de 2020 sont les mêmes que 2019, du coup je n'ai pas utilisé 2020.

Ensuite on ajoute les données de débit d'eau d'il y a 1h, 2h, et 3h.

In [37]:
data.columns= ["Date","Q"]
data["Q-1"] = data["Q"].shift()
data["Q-2"] = data["Q"].shift(periods=2)
data["Q-3"] = data["Q"].shift(periods=3)

How to fill NaN on the first lines?

In [38]:
data

,Date,Q,Q-1,Q-2,Q-3
0,2010-01-01 00:00:00,55.8,NaN,NaN,NaN
1,2010-01-01 01:00:00,56.2,55.8,NaN,NaN
2,2010-01-01 02:00:00,56.6,56.2,55.8,NaN
3,2010-01-01 03:00:00,57.0,56.6,56.2,55.8
4,2010-01-01 04:00:00,57.4,57.0,56.6,56.2
...,...,...,...,...,...
8754,2019-12-31 18:00:00,186.0,185.5,185.0,184.5
8755,2019-12-31 19:00:00,186.5,186.0,185.5,185.0
8756,2019-12-31 20:00:00,187.0,186.5,186.0,185.5
8757,2019-12-31 21:00:00,189.0,187.0,186.5,186.0


In [39]:
for i in range(1,13):
    meteo = pd.read_csv("../Data/meteo/"+ str(i) +".csv")
    data=pd.merge(data,meteo)
    data.rename(columns={'Température': 'Temp'+str(i),
                   'Précipitations dans la dernière heure': 'Prec'+str(i),
                  'Précipitations dans les 3 dernières heures':'Prec3h'+str(i)},
          inplace=True, errors='raise')

In [40]:
data.to_csv('../Data/meteo/H1700010_bind.csv', header = True)